In [199]:
import yfinance as yf
import requests_cache
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import os
import json
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.preprocessing import Normalizer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV
from sklearn.metrics import make_scorer,mean_squared_error
from xgboost import XGBRegressor
from pycoingecko import CoinGeckoAPI
import cryptocompare


# decision tree
from sklearn.tree import DecisionTreeRegressor

from datetime import datetime, timedelta
# random forest
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_regression, SelectFromModel

from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
import ta

# Define the parameter grid
PARAM_GRID = {
    'n_estimators': [100, 200, 400],
    'max_depth': [6, 12, 24],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

RANDOM_PARAM_GRID = {
    'n_estimators': [int(x) for x in range(50, 1000, 50)],
    'max_depth': [None] + [int(x) for x in range(10, 110, 10)],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt']
}

XGB_PARAM_GRID= {
    'n_estimators': [100, 500],
    'learning_rate': [0.01, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.6, 0.8, 1.0],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

BEST_PARAMS = {'max_depth': 12, 'min_samples_leaf': 4, 'min_samples_split': 10, 'n_estimators': 100}

voting_regressor = VotingRegressor([
    ('lgr', LinearRegression()),
    ('hist', HistGradientBoostingRegressor()),
    ('rf', RandomForestRegressor()),
    ('gbm', GradientBoostingRegressor()),
    # ('xgb', XGBRegressor())
])

ACTIVE_MODEL = Pipeline([
    ('Scaler', StandardScaler()),
    # ('FeatureSelection', SelectKBest(score_func=f_regression, k=10)),
    ('Regressor', voting_regressor)
    ]
)
# ACTIVE_MODEL = RandomForestRegressor(**BEST_PARAMS)
# ACTIVE_MODEL=XGBRegressor(**{'subsample': 1.0, 'n_estimators': 100, 'max_depth': 3, 'learning_rate': 0.1, 'colsample_bytree': 0.8})


from dotenv import load_dotenv
load_dotenv()


PERIOD = 365
DAYS_TO_BACKTEST = 4
COINGECKO_KEY = os.getenv('COINGECKO_KEY')
CC_KEY = os.getenv('CC_KEY')
# format to 2 decimal in print
pd.options.display.float_format = '{:.2f}'.format

print('done', COINGECKO_KEY[:3], CC_KEY[:3])


done CG- ba1


In [200]:

class CoinPredict:
  COLS_TO_DROP = [
    'Conversiontype', 'Conversionsymbol', 'Volumefrom', 'Volumeto',
    'Range', 'Volume', 'High', 'Low', 'day of week', 'month', 'Dividends', 'Coin Splits', 'Capital Gains',]
  TARGET_COL = 'target'
  BASE_DIR = 'data'

  def __init__(self, symbol='ETH', period=PERIOD, days_to_backtest=DAYS_TO_BACKTEST, future_close_day_to_predict=1, max_features=14):
    self.symbol = symbol
    self.folder =  f"{CoinPredict.BASE_DIR}/{self.symbol.lower()}"
    self.period = period
    self.future_close_day_to_predict = future_close_day_to_predict
    self.days_to_backtest = days_to_backtest
    # https://github.com/man-c/pycoingecko
    self.cg = CoinGeckoAPI(demo_api_key=COINGECKO_KEY)
    cryptocompare.cryptocompare._set_api_key_parameter(CC_KEY)

    self.selector_model = None
    self.max_features = max_features

  def get_coins_list(self):
    coins = cryptocompare.get_coin_list(format=False)

    # if file exists return
    if os.path.exists("coins.json"):
      print("coins.json already exists")
    else:
      with open("coins.json", "w") as f:
        f.write(json.dumps(coins))

    symbol_file = "coins_short.json"
    symbols = []
    for k, v in coins.items():
      symbols.append({
        'id': v['Id'],
        'symbol': v['Symbol'],
        'name': v['Name']
      })
    with open(symbol_file, "w") as f:
      f.write(json.dumps(symbols))


  def load_data(self):
    now = pd.Timestamp.now()
    current_date_string = now.strftime('%Y-%m-%d')
    data_file = f"{self.folder}/{self.period}_{current_date_string}.csv"
    if not os.path.exists(self.folder):
      # create folder if needed
      os.makedirs(self.folder)

    if os.path.exists(data_file):
      print("loading data from file: " + data_file)
      # read 'Date' as index
      df = pd.read_csv(data_file, header=0, index_col=0)
    else:
      from_timestamp = int((now - pd.Timedelta(days=int(self.period))).timestamp())
      to_timestamp = int(now.timestamp())
      print("loading data from coingecko", from_timestamp, to_timestamp)
      data = cryptocompare.get_historical_price_day(self.symbol, currency='USD',
                                                    limit=365, toTs=to_timestamp)

      # print('history metadata', self.ticker.history_metadata)
      df = pd.DataFrame(data)
      df.index = df['time'].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y-%m-%d %H:%M:%S'))
      # capitalize columns
      df['volume'] = df['volumeto'] - df['volumefrom']
      df.columns = [x.capitalize() for x in df.columns]

    return self.process_data(df)

  def get_last_quote(self):
    if not COINGECKO_KEY:
      return None

    quote = cryptocompare.get_price(self.symbol, currency='USD', full=False)
    quote['c'] = quote[self.symbol.upper()]['USD']
    # now
    quote['t'] = datetime.now().timestamp()

    return quote


  def process_data(self, df):
    df.index = pd.to_datetime(df.index, utc=True)
    # df.index = pd.to_datetime(df.index)

    df['Range'] = df['High'] - df['Low']
    for i in range(1, self.days_to_backtest):
      df[f'close diff {i} days ago'] = df['Close'].shift(i-1) - df['Close'].shift(i)
      df[f'volume {i} days ago'] = df['Volume'].shift(i)
      df[f'range {i} days ago'] = df['Range'].shift(i)

    for i in range(1, self.future_close_day_to_predict):
      df['Close +'+str(i)] = df['Close'].shift(-i)

    df['high yesterday'] = df['High'].shift(1)
    df['low yesterday'] = df['Low'].shift(1)

    # df[CoinPredict.TARGET_COL] =  df['Close'].shift(-1) - df['Close'] # change in close
    df[CoinPredict.TARGET_COL] = df['Close'].shift(-self.future_close_day_to_predict) # next close
    # df[CoinPredict.TARGET_COL] = (df['Close'].shift(-1) - df['Open']).apply(lambda x: 'up' if x > 0 else 'down') # next close direction (up or down)

    # set day of week
    # as datetime
    df['day of week'] = df.index.day_of_week
    df['month'] = df.index.month

    from ta import add_all_ta_features

    df['EMA_5'] = ta.trend.ema_indicator(df['Close'], window=5)
    df['EMA_20'] = ta.trend.ema_indicator(df['Close'], window=20)
    # https://github.com/bukosabino/ta
    df = add_all_ta_features(df, open="Open", high="High", low="Low", close="Close", volume="Volume", fillna=True)

    # one hot encode month and day of week
    # df = pd.get_dummies(df, columns=['day of week', 'month'])

    # drop columns (final step)
    cols_to_drop = CoinPredict.COLS_TO_DROP
    # if last row is missing close, drop it
    if pd.isna(df.iloc[-1]['Close']):
      cols_to_drop.append('Close')

    df = df.drop(cols_to_drop, axis=1, errors='ignore')

#     print('df columns', df.columns)
    # fill nan with mean
    df = df.fillna(df.mean())

    after_close = not pd.isna(df.iloc[-1]['Close'])

    shift_days = self.future_close_day_to_predict - int(after_close)
    df = df[self.days_to_backtest:]

    print('shift', shift_days, after_close)
    if shift_days > 0:
      df = df[:-shift_days]


    if not self.selector_model:
      self.selector_model = RandomForestRegressor()
      self.selector = SelectFromModel(self.selector_model, max_features=self.max_features, threshold=-np.inf) # take top max_features
      # define X and y
      X_train = df.drop([CoinPredict.TARGET_COL], axis=1)
      y_train = df[CoinPredict.TARGET_COL]
      self.selector_model.fit(X_train, y_train)
      self.selector.fit(X_train, y_train)
      # get selected columns

    if hasattr(self.selector, 'get_support'):
      selected_columns = X_train.columns[self.selector.get_support()]
      # keep target
      for col in [CoinPredict.TARGET_COL, 'Open', 'Close']:
        if col not in selected_columns:
          selected_columns = np.append(selected_columns, col)
      print('selected columns', selected_columns)
      df = df[selected_columns]

    last_row = df.tail(1)

    return df[:-1], last_row


  def plot_data(self, df):
    plt.figure(figsize=(16,8))
    plt.title('Close Price History')
    # plot with date
    X = df.index
    y = df['Close']
    plt.plot(X,y)
    plt.xlabel('Date', fontsize=18)
    plt.ylabel('Close Price USD ($)', fontsize=18)
    plt.show()

  def train_model(self, df, debug=False):
    # define X and y
    X_train = df.drop([CoinPredict.TARGET_COL], axis=1)
    self.trained_columns = X_train.columns.values
    if debug:
      print('training shape', X_train.shape)
      print('training columns', X_train.columns.values)
    y_train = df[CoinPredict.TARGET_COL]
    # split data

    # create regressor
    self.model = ACTIVE_MODEL#(**PARAMS)

    # fit model
    self.model.fit(X_train, y_train)
    print('example train rows', y_train[-3:])

  def param_search(self, df, random_search=True):
    if not self.model:
      raise Exception('model not trained')

    # define X and y
    X_train = df.drop([CoinPredict.TARGET_COL], axis=1)
    y_train = df[CoinPredict.TARGET_COL]
    if random_search:
      grid_search = RandomizedSearchCV(estimator=self.model,
                                   param_distributions=XGB_PARAM_GRID,
                                   n_iter=100,
                                   cv=5,
                                   verbose=2,
                                   random_state=42,
                                   n_jobs=-1)
    else:
      grid_search = GridSearchCV(estimator=self.model, param_grid=PARAM_GRID,
                            scoring='neg_mean_absolute_error', cv=3, n_jobs=-1)
    # Perform the grid search
    grid_search.fit(X_train, y_train)

    # Get the best parameters and best score
    best_params = grid_search.best_params_
    best_score = grid_search.best_score_

    print("Best Parameters:", best_params)
    print("Best Score:", best_score)
    return best_params, best_score

  def cross_val_score(self, df, debug=False, cv=4):
    if not hasattr(self, 'model'):
      raise Exception('model not trained')

    X = df.drop([CoinPredict.TARGET_COL], axis=1)
    y = df[CoinPredict.TARGET_COL]
    scores = cross_val_score(self.model, X, y, cv=cv)
    print(f"Symbol: {self.symbol} - Days to compare: {self.days_to_backtest}\ncross val scores\n---\n{scores}")

    # print other stats
    print("Average CV: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

    model = ACTIVE_MODEL

    # test train split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

    # fit model
    model.fit(X_train, y_train)

    # predict
    y_pred = model.predict(X_test)
    mean_abs_error = metrics.mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(metrics.mean_squared_error(y_test, y_pred))

    if debug:
      print('Mean Absolute Error:', mean_abs_error)
      print('Mean Squared Error:', metrics.mean_squared_error(y_test, y_pred))
      print('Root Mean Squared Error:', rmse)
      print('R2 Score:', metrics.r2_score(y_test, y_pred))

    direction = y_pred - X_test['Open']
    direction = direction.apply(lambda x: 'up' if x > 0 else 'down')
    actual_direction = y_test - X_test['Open']
    actual_direction = actual_direction.apply(lambda x: 'up' if x > 0 else 'down')

    # print a few comparisons
    print('Comparing a few predictions side by side...')
    amount_error = y_pred - y_test
    percent_error = amount_error / y_test * 100
    # correct if same direction and error within 1%

    correct = (direction == actual_direction) & (abs(amount_error) <= 2*mean_abs_error)
    accuracy_score = correct.sum() / len(correct) * 100
    print(f"Direction Accuracy using {self.future_close_day_to_predict} days: {accuracy_score}%")
    error_df = pd.DataFrame({'Reference open': X_test['Open'],
                             'Direction': direction,
                              'Correct': correct,
                              'Amount Error': amount_error,
                              'Error %': percent_error,
                              'Predicted': y_pred })
    actual_col_name = f"Close +{self.future_close_day_to_predict} (actual)"
    error_df[actual_col_name] = y_test
    # take last 10 by close
    error_df = error_df.tail(10)
    print(error_df.to_markdown())

    # return average
    return scores.mean(), mean_abs_error

  def get_importance(self):
    if not hasattr(self, 'model'):
      raise Exception('model not trained')

    class_name = self.model.__class__.__name__
    if class_name == 'XGBRegressor':
      importances = self.model.get_booster().get_score(importance_type="gain")
      # sort by importance
      importances = sorted(importances.items(), key=lambda x: x[1], reverse=True)
    # check if pipeline
    elif class_name == 'Pipeline':
      try:
        importances = self.model.named_steps['Regressor'].feature_importances_
        importances = sorted(zip(self.trained_columns, importances), key=lambda x: x[1], reverse=True)
      except Exception as e:
        importances = []
    else:
      try:
        importances = self.model.feature_importances_
        importances = sorted(zip(self.trained_columns, importances), key=lambda x: x[1], reverse=True)
      except Exception as e:
        importances = []

    if not importances:
      print('Using selector model importances')
      try:
        importances = self.selector_model.feature_importances_
        importances = sorted(zip(self.trained_columns, importances), key=lambda x: x[1], reverse=True)
      except Exception as e:
        importances = []
        print('error getting importances', e)

    return importances

  def predict(self, last_row):
    # predict
    X = last_row.drop([CoinPredict.TARGET_COL], axis=1)
    # y = last_row['next_close']
    y_pred = self.model.predict(X)
    return y_pred


In [201]:
cp = CoinPredict(symbol='eth', days_to_backtest=DAYS_TO_BACKTEST, future_close_day_to_predict=5)
cp.get_coins_list()
df, X_test = cp.load_data()
cp.train_model(df, True)
print('\n***\n')

coins.json already exists
loading data from file: data/eth/365_2025-01-19.csv
shift 4 True


/Users/chrisbuonocore/personal/python/coin-predict/env/lib/python3.12/site-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2
/Users/chrisbuonocore/personal/python/coin-predict/env/lib/python3.12/site-packages/ta/wrapper.py:487: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"{colprefix}momentum_ppo"] = indicator_ppo.ppo()
/Users/chrisbuonocore/personal/python/coin-predict/env/lib/python3.12/site-packages/ta/wrapper.py:488: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.i

selected columns ['Time' 'volume 1 days ago' 'range 1 days ago' 'close diff 2 days ago'
 'volume 3 days ago' 'Close +2' 'Close +3' 'Close +4' 'volume_nvi'
 'trend_mass_index' 'trend_kst_sig' 'trend_adx' 'trend_adx_pos'
 'momentum_stoch_rsi_d' 'target' 'Open' 'Close']
training shape (357, 16)
training columns ['Time' 'volume 1 days ago' 'range 1 days ago' 'close diff 2 days ago'
 'volume 3 days ago' 'Close +2' 'Close +3' 'Close +4' 'volume_nvi'
 'trend_mass_index' 'trend_kst_sig' 'trend_adx' 'trend_adx_pos'
 'momentum_stoch_rsi_d' 'Open' 'Close']
example train rows time
2025-01-12 00:00:00+00:00   3475.27
2025-01-13 00:00:00+00:00   3304.42
2025-01-14 00:00:00+00:00   3210.82
Name: target, dtype: float64

***



In [202]:
mean_cv_score, mean_abs_error = cp.cross_val_score(df, True)


Symbol: eth - Days to compare: 4
cross val scores
---
[0.94040282 0.84327381 0.75495221 0.88597922]
Average CV: 0.86 (+/- 0.14)
Mean Absolute Error: 74.81698774826418
Mean Squared Error: 9936.631304926814
Root Mean Squared Error: 99.6826529789753
R2 Score: 0.9587010898473319
Comparing a few predictions side by side...
Direction Accuracy using 5 days: 86.11111111111111%
| time                      |   Reference open | Direction   | Correct   |   Amount Error |   Error % |   Predicted |   Close +5 (actual) |
|:--------------------------|-----------------:|:------------|:----------|---------------:|----------:|------------:|--------------------:|
| 2024-08-20 00:00:00+00:00 |          2637.47 | up          | True      |        31.8771 |  1.16021  |     2779.4  |             2747.52 |
| 2024-04-10 00:00:00+00:00 |          3504.83 | down        | True      |        49.6272 |  1.59933  |     3152.63 |             3103    |
| 2024-04-05 00:00:00+00:00 |          3329.17 | up          | True 

In [203]:
last_quote = cp.get_last_quote()
last_quote


{'ETH': {'USD': 3200.29}, 'c': 3200.29, 't': 1737335550.083531}

In [204]:
X_test

,Time,volume 1 days ago,range 1 days ago,close diff 2 days ago,volume 3 days ago,Close +2,Close +3,Close +4,volume_nvi,trend_mass_index,trend_kst_sig,trend_adx,trend_adx_pos,momentum_stoch_rsi_d,target,Open,Close
time,,,,,,,,,,,,,,,,,
2025-01-15 00:00:00+00:00,1736899200,1982124076.85,131.22,87.89,676336363.29,3475.27,3304.42,3210.82,1575.63,23.79,-57.29,24.06,19.11,0.17,3105.32,3224.35,3451.52


In [205]:

y_pred = cp.predict(X_test)
if mean_abs_error:
  pred_close_date =  X_test.index[0] + pd.Timedelta(f"{cp.future_close_day_to_predict} days")
  # round to 2 decimal
  predicted_value = round(y_pred[0], 2)
  print(f"\n${predicted_value} - Close prediction at {pred_close_date}")

  if last_quote:
    last_price = float(last_quote['c']) # ex: https://www.alphavantage.co/query?function=GLOBAL_QUOTE&symbol=IBM&apikey=demo
    last_price_time_seconds = last_quote['t']
    formatted_time_in_est = pd.to_datetime(last_price_time_seconds, unit='s').tz_localize('UTC').tz_convert('US/Eastern')
    higher_or_lower = 'higher' if predicted_value > last_price else ('lower' if predicted_value < last_price else 'same')
    print(f"${last_price} - Last price: {formatted_time_in_est}\n{higher_or_lower}")
  else:
    last_price = X_test['Open'].iloc[0]
  pred_diff = predicted_value - last_price
  print(f"\nMean abs error: {mean_abs_error}")
  # action
  min_threshold = mean_abs_error * 0.5
  print(f"The predicted change is ${round(pred_diff, 2)}. The minimum threshold for action is ${round(min_threshold, 2)} which is half the mean absolute prediction error.")
  if pred_diff > min_threshold:
    action = 'buy'
  elif pred_diff < -min_threshold:
    action = 'sell'
  else:
    action = 'hold'
  print(f"{action.capitalize()} {cp.symbol}")

# get next weekday (monday, tuesday, etc.) after X_test index - if saturday or sunday use monday
next_weekday = X_test.index + pd.Timedelta('1 day')
if next_weekday.day_of_week in [5, 6]:
  next_weekday = X_test.index + pd.offsets.BDay(1)
next_weekday = next_weekday.strftime('%Y-%m-%d')



$3209.87 - Close prediction at 2025-01-20 00:00:00+00:00
$3200.29 - Last price: 2025-01-19 20:12:30.083530903-05:00
higher

Mean abs error: 74.81698774826418
The predicted change is $9.58. The minimum threshold for action is $37.41 which is half the mean absolute prediction error.
Hold eth
